# 1. Construct an image X with diagonal edges.

In [4]:
import torch

def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w]*K).sum()
    return Y

K = torch.tensor([[1.0,-1.0]])
X = torch.eye(6,8)
X

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.]])

## 1.1 What happens if you apply the kernel K in this section to it?

In [8]:
corr2d(X, K)

tensor([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -1.,  1.,  0.]])

## 1.2 What happens if you transpose X?

In [9]:
corr2d(X.T, K)

tensor([[ 1.,  0.,  0.,  0.,  0.],
        [-1.,  1.,  0.,  0.,  0.],
        [ 0., -1.,  1.,  0.,  0.],
        [ 0.,  0., -1.,  1.,  0.],
        [ 0.,  0.,  0., -1.,  1.],
        [ 0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]])

## 1.3 What happens if you transpose K?

In [7]:
corr2d(X, K.T)

tensor([[ 1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.]])

# 2. Design some kernels manually.

In [263]:
import torch

# Define the directional vector
v = torch.tensor([1, 1], dtype=torch.float32)  # Replace v1 and v2 with your values

# Normalize the directional vector
u = v / torch.norm(v)

# Create the edge-detection kernel along the direction (v2, -v1)
edge_detection_kernel = torch.tensor([[-u[1], u[0]],
                                      [-u[0], -u[1]]], dtype=torch.float32)

# Square the kernel element-wise to enhance edge detection effect
# edge_detection_kernel_squared = edge_detection_kernel ** 2

print("Edge-detection kernel:")
print(edge_detection_kernel)


Edge-detection kernel:
tensor([[-0.7071,  0.7071],
        [-0.7071, -0.7071]])


In [256]:
pic

tensor([[1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.]])

In [257]:
K = torch.tensor([[0,-1],[1,0]])
corr2d(X,K)

tensor([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0., -1.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0., -1.,  0.,  1.,  0.]])

In [238]:
def detect_edge(v,image_tensor):
    theta = stat_theta(v)
    kernel_x = torch.tensor([[1, 0, -1],
                         [2, 0, -2],
                         [1, 0, -1]], dtype=torch.float32)

    kernel_y = torch.tensor([[1, 2, 1],
                             [0, 0, 0],
                             [-1, -2, -1]], dtype=torch.float32)
    angle_radians = torch.deg2rad(torch.tensor(theta))
    rotated_kernel_x = torch.tensor([[torch.cos(angle_radians), -torch.sin(angle_radians), 0],
                                 [torch.sin(angle_radians), torch.cos(angle_radians), 0],
                                 [0, 0, 0]])

    rotated_kernel_y = torch.tensor([[torch.cos(angle_radians), -torch.sin(angle_radians), 0],
                                     [torch.sin(angle_radians), torch.cos(angle_radians), 0],
                                     [0, 0, 0]])
    edge_detected_x = corr2d(image_tensor, rotated_kernel_x)
    edge_detected_y = corr2d(image_tensor, rotated_kernel_y)
    # Combine the x and y edge-detected results to get the final edge-detection result
    edge_detected = torch.sqrt(edge_detected_x ** 2 + edge_detected_y ** 2)
    return edge_detected

r = [2,1]
theta = stat_theta(r)
b = tF.rotate(a.reshape(1,1,a.shape[0],-1,),angle=theta).reshape(a.shape[0],-1)
print(b)
detect_edge(r,b)

tensor([[0., 0., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 0., 0., 0.]])


tensor([[0.6325, 0.0000, 0.0000, 0.0000, 0.6325, 2.5298],
        [3.1623, 0.6325, 0.0000, 0.0000, 0.6325, 1.8974],
        [2.5298, 1.8974, 0.0000, 0.0000, 0.0000, 0.6325],
        [2.5298, 3.1623, 0.6325, 0.0000, 0.0000, 0.6325]])

In [229]:
K=torch.tensor([[0,-1],[1,0]])

In [228]:
pic

tensor([[1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.]])

In [230]:
corr2d(pic,K)

tensor([[-1.,  0.,  1.,  1.,  0.,  0.,  0.],
        [-1., -1.,  0.,  1.,  1.,  0.,  0.],
        [ 0., -1., -1.,  0.,  1.,  1.,  0.],
        [ 0.,  0., -1., -1.,  0.,  1.,  1.],
        [ 0.,  0.,  0., -1., -1.,  0.,  1.]])

In [225]:
angle_degrees = 90
rotated_angle_radians = torch.deg2rad(torch.tensor(angle_degrees))
v = torch.tensor([torch.cos(rotated_angle_radians), torch.sin(rotated_angle_radians)])
print(rotated_angle_radians,v)
# Compute the orthogonal direction
orthogonal_direction = torch.tensor([v[1], -v[0]])

# Create the edge-detection kernel
kernel = torch.outer(orthogonal_direction, orthogonal_direction)
kernel

tensor(1.5708) tensor([-4.3711e-08,  1.0000e+00])


tensor([[1.0000e+00, 4.3711e-08],
        [4.3711e-08, 1.9107e-15]])

In [226]:
corr2d(a,kernel)

tensor([[1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3711e-08,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3711e-08,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3711e-08,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3711e-08,
         1.0000e+00],
        [1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.3711e-08,
         1.0000e+00]])

In [253]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as tF
def stat_theta(r):
    v = [1,0]
    theta = math.acos(F.cosine_similarity(torch.tensor(r).type(torch.float32),torch.tensor(v).type(torch.float32),dim=0))/math.pi*180
    return theta

def gen_K(v):
    u = torch.tensor([[-v[1], v[0]]])
    K = torch.cat((-u,u),dim=0).T
    # K = torch.cat((u,torch.zeros(u.shape),-1*u)).reshape(1,-1)
    return K

r = [1,0]
theta = stat_theta(r)
K = gen_K(r)
print(K)
b = tF.rotate(a.reshape(1,1,a.shape[0],-1,),angle=theta).reshape(a.shape[0],-1)
print(b)
corr2d(b,K)

tensor([[ 0,  0],
        [-1,  1]])
tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [255]:
r = [0,1]
theta = stat_theta(r)
K = gen_K(r)
print(K)
b = tF.rotate(a.reshape(1,1,a.shape[0],-1,),angle=theta).reshape(a.shape[0],-1)
b[0:2,:]=1
b[-2:,:]=1
print(b)
corr2d(b,K)

tensor([[ 1, -1],
        [ 0,  0]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [261]:
r = [1,1]
theta = stat_theta(r)
K = gen_K(r)
print(K)
b = tF.rotate(a.reshape(1,1,a.shape[0],-1,),angle=theta).reshape(a.shape[0],-1)
b[0:2,:]=1
b[-2:,:]=1
print(b)
corr2d(b,K)

tensor([[ 1, -1],
        [-1,  1]])
tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 1., -1.,  0.,  0.,  0., -1.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [169]:
corr2d(a,K.T)

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [133]:
b = F.rotate(a.reshape(1,1,a.shape[0],-1,),angle=30).reshape(a.shape[0],-1)
b

tensor([[0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0.]])

In [147]:
K = torch.zeros(2,3)
K[-1,-1] = 1
K[0,0] = 1
K

tensor([[1., 0., 0.],
        [0., 0., 1.]])

In [149]:
b

tensor([[0., 0., 0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0.]])

In [148]:
corr2d(b,K)

tensor([[0., 0., 0., 1., 1., 2.],
        [0., 0., 0., 0., 1., 2.],
        [1., 0., 0., 0., 0., 1.],
        [2., 1., 0., 0., 0., 0.],
        [2., 1., 1., 0., 0., 0.]])

In [102]:
import torch
import torchvision.transforms.functional as F

# Define the angle of rotation in degrees
angle_degrees = 45

# Convert the angle to radians
angle_radians = torch.deg2rad(torch.tensor(angle_degrees))

# Create the rotation matrix
rotation_matrix = torch.tensor([[torch.cos(angle_radians), -torch.sin(angle_radians)],
                                [torch.sin(angle_radians), torch.cos(angle_radians)]])

# Define the original matrix (points to rotate)
points = torch.tensor([[1, 2],
                       [3, 4],
                       [5, 6]], dtype=torch.float32)

# Apply rotation using matrix multiplication
rotated_points = torch.mm(points, rotation_matrix)

print("Original points:")
print(points)
print("Rotated points:")
print(rotated_points)


Original points:
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])
Rotated points:
tensor([[2.1213, 0.7071],
        [4.9497, 0.7071],
        [7.7782, 0.7071]])


## 2.1 Given a directional vector $\vec{v}=(v_1,v_2)$, derive an edge-detection kernel that detects edges orthogonal to $\vec{v}$, i.e., edges in the direction $(v_2,-v_1)$.

In [99]:
import torch

# Define the directional vector
v = torch.tensor([3, 4], dtype=torch.float32)

# Compute the orthogonal direction
orthogonal_direction = torch.tensor([v[1], -v[0]])

# Create the edge-detection kernel
kernel = torch.outer(orthogonal_direction, orthogonal_direction)

print("Edge-detection kernel:")
print(kernel)


Edge-detection kernel:
tensor([[ 16., -12.],
        [-12.,   9.]])


In [100]:
import torch

# Define the directional vector
v = torch.tensor([3, 4], dtype=torch.float32)

# Compute the orthogonal direction
orthogonal_direction = torch.tensor([v[1], -v[0]], dtype=torch.float32)

# Create the edge-detection kernel
kernel = torch.outer(orthogonal_direction, orthogonal_direction)

# Define the matrix size
matrix_size = 5

# Create the example matrix with the given direction
matrix = torch.zeros((matrix_size, matrix_size), dtype=torch.float32)
for i in range(matrix_size):
    for j in range(matrix_size):
        matrix[i, j] = v.dot(torch.tensor([i, j], dtype=torch.float32))

# Apply the edge-detection kernel using convolution
edge_detected = torch.conv2d(matrix.unsqueeze(0).unsqueeze(0), kernel.unsqueeze(0).unsqueeze(0))

print("Original matrix:")
print(matrix)
print("Edge-detected matrix:")
print(edge_detected.squeeze())


Original matrix:
tensor([[ 0.,  4.,  8., 12., 16.],
        [ 3.,  7., 11., 15., 19.],
        [ 6., 10., 14., 18., 22.],
        [ 9., 13., 17., 21., 25.],
        [12., 16., 20., 24., 28.]])
Edge-detected matrix:
tensor([[-21., -17., -13.,  -9.],
        [-18., -14., -10.,  -6.],
        [-15., -11.,  -7.,  -3.],
        [-12.,  -8.,  -4.,   0.]])


In [87]:
a = torch.ones((6, 8))
a[:, 2:6] = 0
a

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [93]:
K = torch.tensor([[1,-1],[-1,1]])
corr2d(torch.fliplr(X),K)

tensor([[ 0.,  0.,  0.,  0.,  0.,  1., -2.],
        [ 0.,  0.,  0.,  0.,  1., -2.,  1.],
        [ 0.,  0.,  0.,  1., -2.,  1.,  0.],
        [ 0.,  0.,  1., -2.,  1.,  0.,  0.],
        [ 0.,  1., -2.,  1.,  0.,  0.,  0.]])

In [84]:
import torch

# Create a sample matrix A
A = torch.tensor([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]], dtype=torch.float32)

# Create a tensor D with diagonal elements for replacement
D = torch.tensor([10, 20, 30], dtype=torch.float32)

# Replace diagonal elements of A with D
A_diag_replaced = torch.diag_embed(D)

print("Original matrix A:")
print(A)
print("Diagonal-replaced matrix:")
print(A_diag_replaced)


Original matrix A:
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
Diagonal-replaced matrix:
tensor([[10.,  0.,  0.],
        [ 0., 20.,  0.],
        [ 0.,  0., 30.]])


In [68]:
torch.fliplr(pic)

tensor([[0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.]])

In [64]:
K = torch.tensor([[1,-1],[-1,1]])
K

tensor([[ 1, -1],
        [-1,  1]])

In [72]:
corr2d(torch.fliplr(pic),K)

tensor([[ 0.,  0.,  0.,  1., -1.,  0., -1.],
        [ 0.,  0.,  1., -1.,  0., -1.,  1.],
        [ 0.,  1., -1.,  0., -1.,  1.,  0.],
        [ 1., -1.,  0., -1.,  1.,  0.,  0.],
        [-1.,  0., -1.,  1.,  0.,  0.,  0.]])

In [41]:
import torch

# Create a sample 2D matrix
matrix = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]], dtype=torch.float32)

# Define the rotation angle in degrees
angle_degrees = 90

# Convert the angle to radians
angle_radians = torch.deg2rad(torch.tensor(angle_degrees))

# Define the rotation matrix
rotation_matrix = torch.tensor([[torch.cos(angle_radians), -torch.sin(angle_radians)],
                                [torch.sin(angle_radians), torch.cos(angle_radians)]])

# Perform the rotation using matrix multiplication
rotated_matrix = torch.mm(rotation_matrix, matrix)

print("Original matrix:")
print(matrix)
print("Rotated matrix:")
print(rotated_matrix)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x2 and 3x3)

In [37]:
pic = torch.roll(X, shifts=[2],dims=[1])+torch.roll(X, shifts=[1],dims=[1])+X
pic

tensor([[1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.]])

In [40]:
v = [2,2]
K = torch.eye(*v)
if v[0]*v[1] > 0:
    K = torch.fliplr(K)
K

tensor([[0., 1.],
        [1., 0.]])

In [39]:
corr2d(pic,K)

tensor([[1., 2., 1., 1., 0., 0., 0.],
        [1., 1., 2., 1., 1., 0., 0.],
        [0., 1., 1., 2., 1., 1., 0.],
        [0., 0., 1., 1., 2., 1., 1.],
        [0., 0., 0., 1., 1., 2., 1.]])

In [24]:
torch.diag(torch.tensor([1,2,3]), diagonal=0 )

tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])

In [29]:
torch.fliplr(torch.diag(torch.tensor([1,2,3]), diagonal=0 ))

tensor([[0, 0, 1],
        [0, 2, 0],
        [3, 0, 0]])

## 2.2 Derive a finite difference operator for the second derivative. What is the minimum size of the convolutional kernel associated with it? Which structures in images respond most strongly to it?


## 2.3 How would you design a blur kernel? Why might you want to use such a kernel?

## 2.4 What is the minimum size of a kernel to obtain a derivative of order?

# 3. When you try to automatically find the gradient for the Conv2D class we created, what kind of error message do you see?



# 4. How do you represent a cross-correlation operation as a matrix multiplication by changing the input and kernel tensors?